In [1]:
import sys
sys.path.append("..")

In [2]:
import re

from resources.breadth_first_solver.breadth_first_solver import BreadthFirstSolver
from resources.breadth_first_solver.game import Game
from resources.utils import get_puzzle_input

### Part 1


As you see the Elves defend their hot chocolate successfully, you go back to falling through time. This is going to become a problem.

If you're ever going to return to your own time, you need to understand how this device on your wrist works. You have a little while before you reach your next destination, and with a bit of trial and error, you manage to pull up a programming manual on the device's tiny screen.

According to the manual, the device has four registers (numbered 0 through 3) that can be manipulated by instructions containing one of 16 opcodes. The registers start with the value 0.

Every instruction consists of four values: an opcode, two inputs (named A and B), and an output (named C), in that order. The opcode specifies the behavior of the instruction and how the inputs are interpreted. The output, C, is always treated as a register.

In the opcode descriptions below, if something says "value A", it means to take the number given as A literally. (This is also called an "immediate" value.) If something says "register A", it means to use the number given as A to read from (or write to) the register with that number. So, if the opcode addi adds register A and value B, storing the result in register C, and the instruction addi 0 7 3 is encountered, it would add 7 to the value contained by register 0 and store the sum in register 3, never modifying registers 0, 1, or 2 in the process.

In [3]:
ops = []

### Addition

```
addr (add register) stores into register C the result of adding register A and register B.
addi (add immediate) stores into register C the result of adding register A and value B.
```

In [4]:
def addr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] + registers[B]
    return tuple(output)

def addi(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] + B
    return tuple(output)

ops.extend((addr, addi))

### Multiplication

```
mulr (multiply register) stores into register C the result of multiplying register A and register B.
muli (multiply immediate) stores into register C the result of multiplying register A and value B.
```

In [5]:
def mulr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] * registers[B]
    return tuple(output)

def muli(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] * B
    return tuple(output)

ops.extend((mulr, muli))

### Bitwise AND

```
banr (bitwise AND register) stores into register C the result of the bitwise AND of register A and register B.
bani (bitwise AND immediate) stores into register C the result of the bitwise AND of register A and value B.
```

In [6]:
def banr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] & registers[B]
    return tuple(output)

def bani(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] & B
    return tuple(output)

ops.extend((banr, bani))

### Bitwise OR

```
borr (bitwise OR register) stores into register C the result of the bitwise OR of register A and register B.
bori (bitwise OR immediate) stores into register C the result of the bitwise OR of register A and value B.
```

In [7]:
def borr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] | registers[B]
    return tuple(output)

def bori(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    output[C] = registers[A] | B
    return tuple(output)

ops.extend((borr, bori))

### Assignment

```
setr (set register) copies the contents of register A into register C. (Input B is ignored.)
seti (set immediate) stores value A into register C. (Input B is ignored.)
```


In [8]:
def setr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions
    output[C] = registers[A]
    return tuple(output)

def seti(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions
    output[C] = A
    return tuple(output)

ops.extend((setr, seti))

### Greater-than testing

```
gtir (greater-than immediate/register) sets register C to 1 if value A is greater than register B. Otherwise, register C is set to 0.
gtri (greater-than register/immediate) sets register C to 1 if register A is greater than value B. Otherwise, register C is set to 0.
gtrr (greater-than register/register) sets register C to 1 if register A is greater than register B. Otherwise, register C is set to 0.
```

In [9]:
def gtir(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    
    output[C] = int(A > registers[B])
    return tuple(output)

def gtri(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    
    output[C] = int(registers[A] > B)
    return tuple(output)

def gtrr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    
    output[C] = int(registers[A] > registers[B])
    return tuple(output)

ops.extend((gtir, gtri, gtrr))

### Equality testing

```
eqir (equal immediate/register) sets register C to 1 if value A is equal to register B. Otherwise, register C is set to 0.
eqri (equal register/immediate) sets register C to 1 if register A is equal to value B. Otherwise, register C is set to 0.
eqrr (equal register/register) sets register C to 1 if register A is equal to register B. Otherwise, register C is set to 0.
```

In [10]:
def eqir(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    
    output[C] = int(A == registers[B])
    return tuple(output)

def eqri(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    
    output[C] = int(registers[A] == B)
    return tuple(output)

def eqrr(registers, instructions):
    output = list(registers)
    _, A, B, C = instructions 
    
    output[C] = int(registers[A] == registers[B])
    return tuple(output)

ops.extend((eqir, eqri, eqrr))

In [11]:
[op.__name__ for op in ops]

['addr',
 'addi',
 'mulr',
 'muli',
 'banr',
 'bani',
 'borr',
 'bori',
 'setr',
 'seti',
 'gtir',
 'gtri',
 'gtrr',
 'eqir',
 'eqri',
 'eqrr']

In [12]:
def probe(before, instructions, after):
    poss_ops  = [op for op in ops if op(before, instructions) == after]
    return poss_ops

In [13]:
assert addi((3, 2, 1, 1), (9, 2, 1, 2)) == (3, 2, 2, 1)

In [14]:
 assert {op.__name__ for op in probe((3, 2, 1, 1), (9, 2, 1, 2), (3, 2, 2, 1))} == {'addi', 'mulr', 'seti'}

Input like:

```
Before: [3, 2, 1, 1]
9 2 1 2
After:  [3, 2, 2, 1]

Before: [3, 2, 1, 1]
9 2 1 2
After:  [3, 2, 2, 1]
...
```

In [15]:
def parse_probes(lines):
    re_before = re.compile(r'^Before: \[([-0-9]+), ([-0-9]+), ([-0-9]+), ([-0-9]+)\]')
    re_after = re.compile(r'^After:  \[([-0-9]+), ([-0-9]+), ([-0-9]+), ([-0-9]+)\]')
    re_inst = re.compile(r'^([-0-9]+) ([-0-9]+) ([-0-9]+) ([-0-9]+)')
    
    state = 'BEFORE'
    probes = []
    
    for line in lines:
        if state == 'BEFORE':
            if re_inst.match(line):
                return probes
            
            match = re_before.match(line)
            if not match:
                raise ValueError('Not BEFORE line: {}'.format(line))
            before = tuple(int(i) for i in match.groups())
            state = 'INST'

        elif state == 'INST':            
            match = re_inst.match(line)
            if not match:
                raise ValueError('Not INST line: {}'.format(line))
            inst = tuple(int(i) for i in match.groups())
            state = 'AFTER'
            
        elif state == 'AFTER':            
            match = re_after.match(line)
            if not match:
                raise ValueError('Not AFTER line: "{}"'.format(line))
            after = tuple(int(i) for i in match.groups())
            probes.append((before, inst, after))
            state = 'BEFORE'
           

In [16]:
puzzle_input = get_puzzle_input('/tmp/day_16.txt')

In [17]:
tests = parse_probes(puzzle_input)

In [18]:
print(len(tests))
tests[0:10]

782


[((3, 3, 2, 3), (3, 1, 2, 2), (3, 3, 2, 3)),
 ((1, 3, 0, 1), (12, 0, 2, 3), (1, 3, 0, 0)),
 ((0, 3, 2, 0), (14, 2, 3, 0), (1, 3, 2, 0)),
 ((2, 3, 3, 3), (10, 0, 3, 0), (2, 3, 3, 3)),
 ((0, 1, 2, 0), (7, 1, 2, 3), (0, 1, 2, 0)),
 ((3, 1, 2, 0), (7, 1, 2, 2), (3, 1, 0, 0)),
 ((1, 2, 1, 3), (6, 2, 2, 2), (1, 2, 2, 3)),
 ((1, 3, 2, 3), (3, 3, 2, 3), (1, 3, 2, 2)),
 ((0, 1, 2, 0), (8, 0, 0, 1), (0, 0, 2, 0)),
 ((1, 2, 3, 1), (0, 2, 3, 2), (1, 2, 1, 1))]

In [19]:
score = 0
for test in tests:
    valid_ops = probe(*test)
    if len(valid_ops) >= 3:
        score += 1
        
score

590

### Part 2

Using the samples you collected, work out the number of each opcode and execute the test program (the second section of your puzzle input).

What value is contained in register 0 after executing the test program?

In [20]:
def possible_ops(tests, max_tests=0):
    possible_matches = []

    # initialise to any index can be any instuction
    for _ in range(len(ops)):
        possible_matches.append(set(ops))

    num_tests = 0
    for before, inst, after in tests:
        op_num = inst[0]
        
        matched_ops = probe(before, inst, after)
        possible_matches[op_num] &= set(matched_ops)
        
        num_tests += 1
        if max_tests and num_tests > max_tests:
            break
        
    return possible_matches

In [21]:
ops_passing_tests = possible_ops(tests)

In [22]:
for i in range(len(ops_passing_tests)):
    names = [op.__name__ for op in ops_passing_tests[i]]
    print('Index {} ops: {}'.format(i, names))

Index 0 ops: ['gtir', 'gtrr', 'eqri']
Index 1 ops: ['gtri', 'bani', 'setr', 'bori', 'eqir', 'banr', 'gtrr', 'muli', 'gtir', 'mulr', 'addi', 'eqrr']
Index 2 ops: ['mulr', 'addi', 'addr', 'muli']
Index 3 ops: ['banr', 'bani']
Index 4 ops: ['gtri', 'bani', 'eqir', 'banr', 'gtrr', 'eqri', 'seti', 'gtir', 'eqrr']
Index 5 ops: ['bani', 'gtrr', 'eqri', 'gtir', 'eqrr']
Index 6 ops: ['muli', 'addr', 'seti']
Index 7 ops: ['gtri', 'bani', 'eqir', 'banr', 'gtrr', 'eqri', 'gtir', 'eqrr']
Index 8 ops: ['gtri', 'muli', 'bani', 'setr', 'addr', 'borr', 'banr', 'gtrr', 'seti', 'gtir', 'mulr', 'addi', 'bori']
Index 9 ops: ['gtir', 'setr']
Index 10 ops: ['banr', 'bani', 'setr']
Index 11 ops: ['bani', 'eqir', 'gtrr', 'eqri', 'gtir', 'eqrr']
Index 12 ops: ['gtri', 'bani', 'banr', 'eqri', 'seti', 'gtir', 'mulr', 'eqrr']
Index 13 ops: ['gtri', 'muli', 'bani', 'setr', 'eqir', 'banr', 'gtrr', 'eqri', 'seti', 'gtir', 'mulr', 'eqrr']
Index 14 ops: ['gtir', 'gtrr']
Index 15 ops: ['banr']


```
15 <-> banr
3  <-> bani
```

In [23]:
#reverse_lookup 
for op in ops:
    poss_idxs = []
    for i in range(len(ops_passing_tests)):
        if op in ops_passing_tests[i]:
            poss_idxs.append(i)
    print(op.__name__, poss_idxs)
        

addr [2, 6, 8]
addi [1, 2, 8]
mulr [1, 2, 8, 12, 13]
muli [1, 2, 6, 8, 13]
banr [1, 3, 4, 7, 8, 10, 12, 13, 15]
bani [1, 3, 4, 5, 7, 8, 10, 11, 12, 13]
borr [8]
bori [1, 8]
setr [1, 8, 9, 10, 13]
seti [4, 6, 8, 12, 13]
gtir [0, 1, 4, 5, 7, 8, 9, 11, 12, 13, 14]
gtri [1, 4, 7, 8, 12, 13]
gtrr [0, 1, 4, 5, 7, 8, 11, 13, 14]
eqir [1, 4, 7, 11, 13]
eqri [0, 4, 5, 7, 11, 12, 13]
eqrr [1, 4, 5, 7, 11, 12, 13]


In [24]:
class OperationMapper(Game):   
    def __init__(self, ops_passing_tests):
        self.ops_passing_tests = ops_passing_tests
        self.num_ops = len(ops_passing_tests)

    @property
    def initial_state(self):
        return ()

    @property
    def ordered_moves(self):
        return False

    def next_moves(self, state):
        mapped = dict(state)
        mapped_idxs = set(mapped.keys())
        mapped_ops = set(mapped.values())
        
        shortest_idx = None
        shortest_ops = None
        for idx in range(self.num_ops):
            if idx in mapped_idxs:
                continue

            remaining_ops = set(self.ops_passing_tests[idx])
            remaining_ops -= mapped_ops
            
            if shortest_idx is None or len(remaining_ops) < len(shortest_ops):
                shortest_idx = idx
                shortest_ops = remaining_ops
                if not shortest_ops:
                    # We have an index that can't be mapped
                    # No valid further mapping
                    break
            
        return {(shortest_idx, op) for op in shortest_ops} 

    def next_state(self, state, move):
        """Next state of the system given move `move` in state `state`."""
        return state + (move,)

    def is_end_state(self, state):
        """Is the current state `state` an end state of the game."""
        return len(state) == self.num_ops

In [25]:
op_mapper = OperationMapper(ops_passing_tests)
solver = BreadthFirstSolver()
solution = solver.solve(op_mapper)
solution

Move 0: examining 1 positions.
Move 1: examining 1 positions.
Move 2: examining 1 positions.
Move 3: examining 1 positions.
Move 4: examining 1 positions.
Move 5: examining 1 positions.
Move 6: examining 1 positions.
Move 7: examining 1 positions.
Move 8: examining 1 positions.
Move 9: examining 1 positions.
Move 10: examining 1 positions.
Move 11: examining 1 positions.
Move 12: examining 1 positions.
Move 13: examining 1 positions.
Move 14: examining 1 positions.
Move 15: examining 1 positions.
Move 16: examining 1 positions.


{((15, <function __main__.banr(registers, instructions)>),
  (3, <function __main__.bani(registers, instructions)>),
  (10, <function __main__.setr(registers, instructions)>),
  (9, <function __main__.gtir(registers, instructions)>),
  (14, <function __main__.gtrr(registers, instructions)>),
  (0, <function __main__.eqri(registers, instructions)>),
  (5, <function __main__.eqrr(registers, instructions)>),
  (11, <function __main__.eqir(registers, instructions)>),
  (7, <function __main__.gtri(registers, instructions)>),
  (4, <function __main__.seti(registers, instructions)>),
  (12, <function __main__.mulr(registers, instructions)>),
  (13, <function __main__.muli(registers, instructions)>),
  (6, <function __main__.addr(registers, instructions)>),
  (2, <function __main__.addi(registers, instructions)>),
  (1, <function __main__.bori(registers, instructions)>),
  (8, <function __main__.borr(registers, instructions)>))}

In [26]:
op_map = dict(list(solution)[0])

In [27]:
part2_input = get_puzzle_input('/tmp/day_16_part_2.txt')

In [28]:
part_2_instructions = []
for line in part2_input:
    part_2_instructions.append(tuple(int(i) for i in line.split(' ')))

In [29]:
part_2_instructions[-5:]

[(2, 1, 3, 1), (4, 2, 3, 2), (7, 2, 1, 3), (13, 3, 2, 3), (6, 0, 3, 0)]

In [30]:
def solve(instructions, op_map):
    registers = (0, 0, 0, 0)
    
    for step in instructions:
        op = op_map[step[0]]
        registers = op(registers, step)
        
    return registers

In [31]:
solve(part_2_instructions, op_map)

(475, 3, 2, 2)